<a href="https://colab.research.google.com/github/ShaunakSen/problem-solving-with-code/blob/master/SQL_Challenge_solns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
/*
Enter your query here.
Please append a semicolon ";" at the end of the query
*/

# STEP 1: RANK OVER NAME, PHONE, AGE, duplicate records get same rank
WITH CTE AS (

SELECT NAME, PHONE, AGE,

RANK() OVER(ORDER BY NAME, PHONE, AGE) as rank_id
FROM EMPLOYEE
),

# STEP 2: extract records having the same rank multiple times - these are the duplicate ones

CTE2 AS (
SELECT NAME, PHONE, AGE

FROM CTE

GROUP BY NAME, PHONE, AGE

HAVING COUNT(rank_id) > 1

)

# STEP 3: extract the duplicate names

SELECT DISTINCT NAME FROM CTE2;

In [ ]:
/*
Enter your query below.
Please append a semicolon ";" at the end of the query
*/

# STEP 1: create a calculated col for year and join with users. also filter for "2019", "2020", "2021"
WITH CTE AS (
SELECT user_name, CONCAT("20", LEFT(transaction_id,2)) AS transaction_year
FROM TRANSACTIONS t

INNER JOIN USERS u

ON t.user_id = u.user_id
WHERE CONCAT("20", LEFT(transaction_id,2)) IN ("2019", "2020", "2021")
),

# STEP 2: use case when to create a calculate col representing each year

CTE2 AS

(
    SELECT user_name,
    CASE WHEN transaction_year = "2019" THEN 1
    ELSE 0 END AS is_2019,
    CASE WHEN transaction_year = "2020" THEN 1
    ELSE 0 END AS is_2020,
    CASE WHEN transaction_year = "2021" THEN 1
    ELSE 0 END AS is_2021,
    
    transaction_year
    FROM CTE
    
)

# STEP 3: sum up each year column to get no of transactions for each user for each year

SELECT user_name,
SUM(is_2019), SUM(is_2020), SUM(is_2021)
FROM CTE2
GROUP BY user_name



In [ ]:
/*
Enter your query below.
Please append a semicolon ";" at the end of the query
*/

# STEP1 : create cols for prev1, prev2 , next1 and next2: these will be used to check if the consecutive periods are high attendance or not


WITH CTE AS (
SELECT id, event_date,
no_of_visitors,
LAG(no_of_visitors, 1) OVER() AS prev_1,
LAG(no_of_visitors, 2) OVER() AS prev_2,
LEAD(no_of_visitors, 1) OVER() AS next_1,
LEAD(no_of_visitors, 2) OVER() AS next_2
FROM  visitors

),

# STEP 2: The no of visitors in that date and the immediate consecutive(prev or next) should be > 100 - the case when statement calculates that column

CTE2 AS (

SELECT
id, event_date,
no_of_visitors,
CASE WHEN (no_of_visitors > 100 and (prev_1 > 100 and prev_2 > 100)) or (no_of_visitors > 100 and (prev_1 > 100 and next_1 > 100)) or (no_of_visitors > 100 and (next_1 > 100 and next_2 > 100)) THEN 1
ELSE 0 END AS is_high_attendance
FROM CTE

)

# STEP3 : extract the rows with high attendance

SELECT 
id, event_date,
no_of_visitors
FROM CTE2
WHERE is_high_attendance = 1